# CSCI 3155 Spring 2024

# Recitation Week 4

## Mython
Today we will work through building a mini-interpreter for a language inspired by Python 3 that we'll call 'Mython'.
You'll explore some design decisions Python makes, and document langauge features in english prose.
You'll then implement a simple interpreter for these features.

### Checking Python 3's behavior
Our class environment on https://coding.csel.io has a Python 3 interpreter installed. You can open up a launcher tab and start a Python 3 console to test out how expressions work in Python 3.


# Mython Language
The grammar for our Mython language is given as follows: 

$$
\begin{array}{rrll}
e & \Rightarrow & n & \texttt{n is a number} \\
&|& s & \texttt{s is a string} \\
&|& (e) \\
&|& e_{left}\; +\; e_{right} \\
&|& e_{left}\; *\; e_{right} \\
\end{array}
$$

Our language supports numbers, strings, a plus operator (`+`), a star operator (`*`), and parentheses. `+` and `*` correspond to addition and multiplication for numbers, but what should its behavior be for strings? What about when one operand is a number, and one operand is a string? For example, what should 'Hello' + 3 do? What about 'Hello * 3? Try a few combinations of these to see what you find!


## Mython Inference Rules
$\newcommand{\llbracket}{[\![}$
$\newcommand{\rrbracket}{]\!]}$
$\newcommand{\denote}[1]{\llbracket #1 \rrbracket}$
$\newcommand{\bigstep}[2]{#1 \Downarrow #2}$
$\newcommand{\eval}[2]{\mathbf{eval}(#1) = #2}$
$\newcommand{\error}{\mathbf{error}}$
$\newcommand{\Number}{\mathbb{R}}$
$\newcommand{\String}{\mathbb{S}}$

Now that you've seen how Python behaves with these operators, lets write some rules for the `+` and `*` operators using english statements. Below we give rules for numbers, you can follow the same format for other cases. Feel free to write these out (on paper, tablet, whiteboard, etc.) instead of typing out the LaTeX notation as done below.

Note that we'll use the shorthand $\eval{e}{n}$ to mean "$e$ evaluates to the value $n$", $n \in \Number$ to mean "$n$ is a (real) number", and $s \in \String$ to mean "$s$ is a string".

**`+` operator rule: two numbers:**

If $\eval{e_{left}}{n_l}$ and $\eval{e_{right}}{n_r}$ and $n_l \in \Number$ and $n_r \in \Number$ then $\eval{e_{left} + e_{right}}{n_l + n_r}$.

**`+` operator rule: two strings:**

If $\eval{e_{left}}{s_1}$ and $\eval{e_{right}}{s_2}$ and $s_1 \in \String$ and $s_2 \in \String$ then $\eval{e_{left} + e_{right}}{s_1s_2}$

**`+` operator rule: number & string:**

Throw error

**`+` operator rule: string & number:**

Throw error

**`*` operator rule: two numbers:**

If $\eval{e_{left}}{n_l}$ and $\eval{e_{right}}{n_r}$ and $n_l \in \Number$ and $n_r \in \Number$ then $\eval{e_{left} * e_{right}}{n_l * n_r}$.

**`*` operator rule: two strings:**

Throw error

**`*` operator rule: number & string:**

If $\eval{e_{left}}{n_1}$ and $\eval{e_{right}}{s_1}$ and $n_1 \in \Number$ and $s_1 \in \String$ then $\eval{e_{left} * e_{right}}{n_1 * s_1}$

**`*` operator rule: string & number:**

If $\eval{e_{left}}{s_1}$ and $\eval{e_{right}}{n_1}$ and $s_1 \in \String$ and $n_1 \in \Number$ then $\eval{e_{left} * e_{right}}{n_1 * s_1}$

# Mython Interpreter

We'll now write a simple interpreter (the `eval` function below) that evaluates Mython expressions using the rules we came up with above. But we don't want to deal with parsing out expressions, so we'll instead use the following Abstract Grammar and just operate on Abstract Syntax Trees (ASTs). Note that we no longer need parentheses in our abstract grammar since an AST has an unambigious order of operations.

### Abstract Grammar
$$
\begin{array}{rrll}
Expr & \Rightarrow & Const(n) & \texttt{n is a number} \\
&|& StrExpr(s) & \texttt{s is a string} \\
&|& Plus(Expr_{left},\; Expr_{right}) \\
&|& Mult(Expr_{left},\; Expr_{right}) \\
\\
\\
Value & \Rightarrow & Num(n) & \texttt{n is a number} \\
&|& Str(s) & \texttt{s is a string} \\
\end{array}
$$


The definitions for an equivalent Abstract Syntax Tree are given for you to use below.

### Abstract Syntax Tree Implementation

In [2]:
sealed trait Expr
case class Const(n: Double) extends Expr
case class StrExpr(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr

sealed trait Value
case class Num(n: Double) extends Value
case class Str(s: String) extends Value

defined trait Expr
defined class Const
defined class StrExpr
defined class Plus
defined class Mult
defined trait Value
defined class Num
defined class Str

### Interpreter

Fill in the interpreter's `eval` implementation with the rules we discussed above. The following cell provides a few test cases you can use to make sure your implementation works. What other tests would you want to write to make sure your `eval` implementation works?

In [3]:
def eval( e: Expr ): Value = {  
    e match {
        case Const(n) => {
            // YOUR CODE HERE
            Num(n)
        }
        case StrExpr(s) => {
            // YOUR CODE HERE
            Str(s)
        }
        case Plus(e1, e2) => {
            // YOUR CODE HERE
            val v1 = eval(e1)
            val v2 = eval(e2)
            (v1, v2) match {
                case (Num(v1), Num(v2)) => Num(v1+v2)
                case (Str(v1), Str(v2)) => Str(v1.concat(v2))
                case (Num(v1), Str(v2)) => throw new IllegalArgumentException("No :(")
                case (Str(v1), Num(v2)) => throw new IllegalArgumentException("No :(")
            }
        }
        case Mult(e1, e2) => {
            // YOUR CODE HERE
            val v1 = eval(e1)
            val v2 = eval(e2)
            (v1, v2) match {
                case (Num(v1), Num(v2)) => Num(v1*v2)
                case (Str(v1), Str(v2)) => throw new IllegalArgumentException("No :(")
                case (Num(v1), Str(v2)) => Str(repeat(v2, v1))
                case (Str(v1), Num(v2)) => Str(repeat(v1, v2))
            }
        }
    }
}


import scala.annotation.tailrec

// Helper function to repeatedly concatenate a string `s` to itself `times` times.
def repeat(s: String, times: Double) : String = {
    @tailrec def helper(n: Double, acc: String) : String = {
        if (n <= 0) {
            acc
        } else {
            helper(n - 1, acc.concat(s))
        }
    };
    helper(times, "")
}

defined function eval
import scala.annotation.tailrec
defined function repeat

In [4]:
// Some tests are provided for you. Add your own to test other cases. Wrap your tests in braces if you want to re-use variable names.
def test_eval(e: Expr, vExpected: Value, testName: String) = {
    val vFound: Value = eval(e)
    assert(vExpected == vFound, s"Test '$testName' failed, expected: $vExpected, got: $vFound")
}

def test_eval_error(e: Expr, testName: String) = { 
    val failed =
        try {
            eval(e)
            true
        } catch {
            case err => false
        }

    if (failed) {
        assert(false, s"Test '$testName' failed, expected an error that did not occur")
    }
}

{
    // 3 + 2
    val e: Expr = Plus(Const(3.0), Const(2.0))
    val vExpected: Value = Num(5.0)
    test_eval(e, vExpected, "Numeric Addition")
}

{
    // "hi" + "bye"
    val e: Expr = Plus(StrExpr("hi"), StrExpr("bye"))
    val vExpected: Value = Str("hibye")
    test_eval(e, vExpected, "String Concatenation")
}

{
    // "hi" * 5
    val e: Expr = Mult(StrExpr("hi"), Const(5.0))
    val vExpected: Value = Str("hihihihihi")
    test_eval(e, vExpected, "Multiply String by Number")
}

{
    // 3 + "hi"
    val e: Expr = Plus(Const(3.0), StrExpr("hi"))
    test_eval_error(e, "Cannot Add Number and String")
}

{
    // 1 + (4 * 2)
    val e: Expr = Plus(Const(1.0), Mult(Const(4.0), Const(2.0)))
    val vExpected: Value = Num(9.0)
    test_eval(e, vExpected, "Multiply then Add")
}

// YOUR TESTS HERE

// END YOUR TESTS
println("All tests passed!")

cmd4.sc:13: This catches all Throwables. If this is really intended, use `case err : Throwable` to clear this warning.
            case err => false
                 ^


All tests passed!


defined function test_eval
defined function test_eval_error